# Example: Bootstrapping

**Execute this notebook in `contre/example`.**

Statistical fluctuations in the Data sample lead to systematic uncertainties in the reweighted MC sample. This uncertainty can be determined by bootstrapping:

The train sample is *resampled*. Different trainings are performed on the resampled train samples.

**About this notebook:**  
The first part consists of the import of the samples and the weights. Afterwards the bootstrapping is started as a `b2luigi` task and the standard devaiation is calculated from the different trainings.

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from root_pandas import read_root

## Imports from `example.ipynb`
The following cells import and redo the reweighting in the example.

### Import samples in `example_input/`

In [ ]:
size_mc=5e6
size_data=1e5
size_mc_offres=15e5
size_data_offres=8e4
frac_a=0.8

In [ ]:
# Scaling the MC to match the data
w = size_data / size_mc
w_offres = size_data_offres / size_mc_offres

In [ ]:
data = read_root("example_input/data.root")
componentA = read_root("example_input/componentA.root")
componentB = read_root("example_input/componentB.root")
data_offres = read_root("example_input/data_offres.root")
componentA_offres = read_root("example_input/componentA_offres.root")

### Import test samples and weights

In [ ]:
with open("example_output/name=my_example/results.json", "r") as f:
    results = json.load(f)
weights = read_root(results["weights"])

In [ ]:
with open("example_output/name=my_example/validation_results.json", "r") as f:
    validation_results = json.load(f)
test_samples = [read_root(sample) for sample in validation_results["test_samples"]]
validation_weights = read_root(validation_results["validation_weights"])

In [ ]:
test_samples = []
test_sample_list = validation_results["test_samples"]

data_offres_test = read_root(test_sample_list[0])
componentA_offres_test = read_root(test_sample_list[1])

In [ ]:
a = validation_weights[len(data_offres_test):]
a = a['weight'].values
componentA_offres_test["contre_weight"] = a

b = weights
b = b['weight'].values
componentA["contre_weight"] = b

In [ ]:
# Rescaling of the on-res. reweighted sample
componentA["contre_weight"] *= size_data / size_data_offres * size_mc_offres / size_mc

## Bootstrapping

The example includes the following steps:

1. Start a set of n trainings from a runfile
2. Import the results
3. Histogram from the n sets of weights
4. Calculate the standard deviation


### Start the training

In [ ]:
%run run_bootstrapping.py

### Import the results

In [ ]:
with open("example_output/name=my_example/bootstrap_results.json") as result_file:
    bootstrap_results = json.load(result_file)

In [ ]:
for i, weight_file in enumerate(bootstrap_results["validation_weights_list"]):
    c = read_root(weight_file)
    c = c["weight"][len(data_offres_test):].values
    componentA_offres_test["weight_"+str(i)] = c
    
for i, weight_file in enumerate(bootstrap_results["weights_list"]):
    c = read_root(weight_file)
    c = c["weight"].values
    componentA["weight_"+str(i)] = c

### Calculate Histograms from the n sets of weights

In [ ]:
variable="variable1"

In [ ]:
offres_histogram_list = []
for i in range(10):
    offres_histogram_list.append(np.histogram(
        componentA_offres_test[variable], bins=30, range=(0,1),
        weights=componentA_offres_test["weight_"+str(i)])[0])
# transpose list of histograms to list of bins
offres_bin_list = np.transpose(offres_histogram_list)
offres_stds = []
for b in offres_bin_list:
    offres_stds.append(np.std(b))

In [ ]:
histogram_list = []
for i in range(10):
    histogram_list.append(np.histogram(
        componentA[variable], bins=30, range=(0,1),
        weights=componentA["weight_"+str(i)])[0])
# transpose list of histograms to list of bins
bin_list = np.transpose(histogram_list)
stds = []
for b in bin_list:
    stds.append(np.std(b))

In [ ]:
# Scale the standart deviation (analouges to the scaling of the on-res. weights)
stds = np.array(stds) * size_data / size_data_offres * size_mc_offres / size_mc

### Plot the result

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=[12.8, 4.8])

# on-resonance histogram
count, edges = np.histogram(
    data[variable], bins=30, range=(0, 1))

bin_width = (edges[1] - edges[0]) / 2
bin_mids = edges[:-1]+bin_width
ax[0].plot(
    bin_mids, count, color="black", marker='.', ls="",
    label="data")

w = size_data/size_mc
mc_count, edges, patches = ax[0].hist(
    [componentA[variable], componentB[variable]],
    bins=30, range=(0, 1), stacked=True,
    weights=[componentA["contre_weight"], [w]*len(componentB)],
    label=["componentA\n(reweighted)", "componentB"])

ax[0].bar(bin_mids, bottom=mc_count[0], height=stds, width=1/30, color="red", label="sys. unc.")


ax[0].set_title("On resonance")
ax[0].legend()

# off-resonance histogram
count, edges = np.histogram(
    data_offres_test[variable], bins=30, range=(0, 1))
ax[1].plot(
    bin_mids, count, color="black", marker='.', ls="")

mc_count, edges, patches = ax[1].hist(
    componentA_offres_test[variable], bins=30, range=(0, 1),
    weights=componentA_offres_test["contre_weight"],
)

ax[1].bar(bin_mids, bottom=mc_count, height=offres_stds, width=1/30, color="red", label="sys. unc.")

ax[1].set_title("Off resonance, test samples")

plt.show()